In [3]:
import requests, json, time, os
from collections import Counter
from IPython.core.display import display, HTML

In [4]:
pathToBattles = r"~\AppData\LocalLow\Sky Mavis\Axie Infinity\production-battleHistoriesState"
pathToBattles = os.path.expanduser(pathToBattles)

In [13]:
with open(pathToBattles, "rt") as f:
    data = json.load(f)
battles = data["battles"]

teams = {}
for battle in battles:
    if battle["battleType"] == 1:
        first = battle["firstClientId"]
        second = battle["secondClientId"]
        enemyNo = first == my and  1 or 0
        enemyId = first == my and second or first
        team = []
        for fighter in battle["fighters"]:
            if fighter["teamIndex"] == enemyNo:
                team.append(fighter["id"])
        teams[enemyId] = team

In [14]:
#for enemy, team in teams.items():
#    print(enemy, team)

In [7]:
from python_graphql_client import GraphqlClient
client = GraphqlClient(endpoint="https://graphql-gateway.axieinfinity.com/graphql")

In [8]:
def getProfileData(address):
    query = """query GetProfileByRoninAddress($roninAddress: String!) {\n  publicProfileWithRoninAddress(roninAddress: $roninAddress) {\n    ...Profile\n    __typename\n  }\n}\n\nfragment Profile on PublicProfile {\n  accountId\n  name\n  addresses {\n    ...Addresses\n    __typename\n  }\n  __typename\n}\n\nfragment Addresses on NetAddresses {\n  ethereum\n  tomo\n  loom\n  ronin\n  __typename\n}\n"""
    variables = {"roninAddress": address}
    data = client.execute(query=query, variables=variables)
    return data

In [9]:
mmr_addr = "https://game-api.axie.technology/mmr/"
addresses = list(teams.keys())
print(len(addresses))
answer = []
for i in range(len(addresses)//10+1):
    mmr_addrs = addresses[i*10:i*10+10]
    if len(mmr_addrs) > 0:
        mmr_req = mmr_addr + ",".join(mmr_addrs)
        print(mmr_req)
        #try:
        data = requests.get(mmr_req).json()
        for result in data:
            if "items" in result:
                for item in result["items"]:
                    if item["client_id"] in mmr_addrs:
                        client_id = item["client_id"]
                        elo = item["elo"]
                        rank = item["rank"]
                        name = item["name"]
                        answer.append([client_id, elo, rank, name])
    time.sleep(3.0)

59
https://game-api.axie.technology/mmr/0x8d7bd51955aefbd827f056080867e7c9e4c38d96,0xef66fa999f752954a1be77e3f0e606fae8a52be0,0xd2c80a65addcb36780d430308b85dd150bbdec4e,0x636605669aed5ba0038b945531cb7fbde6548139,0x144ec2596f7f721b295b07a1941cbb9f3902dcc5,0x99b6bbbdabad08a7632ad0f8bc8cac343c45362a,0x1603e5e3406854849ddbb6157a80cc5abf5cfeb4,0x641b5881f2fc2594270b0e94ff1d2c60e6ac2d2b,0x00e5196ee32ae3e68a618fa8e2f89f3b71fb46ad,0xd224a41143cb7ef4d5f5c68baa205bdd908d8d83
https://game-api.axie.technology/mmr/0x3f33415fc0f800d5643016bddd56e415458fc5af,0xa03a27dba8ee39b651fcd8757bb0025796cde17f,0xdce43a58047d16db5f5bed1c37d03d3d91adf6c0,0x0fa3f1e6e76c1f14a6c3445d46f290c7e4517464,0x669e1df219b7f97119f27cbebc73f30536b7e595,0x3611b49725ae3c4683ac46bceb44cf2941d5051b,0x287be77d593a9f49efc8ac0bb77143585c6b8c02,0x8166b9a7368dd3fa25b077d8253b11d733e5d46a,0x820903d5a4b3b157f656604988a85956272033e1,0x0adb19e7c2b22c3c388f6eb1bd388cee8a4c277d
https://game-api.axie.technology/mmr/0x04eadbc95b778c8e909838ba

In [10]:
answer.sort(key = lambda x: -x[1])

In [11]:
example_link = "https://marketplace.axieinfinity.com/axie/"
def displayAxies(axieList, axieImages, displayLink):
    lines = []
    for axie, img in zip(axieList, axieImages):
        link = example_link + str(axie)
        if displayLink:
            print(link)
        lines.append('<a href="%s" target="_blank" rel="noopener noreferrer"><img src="%s" width=128 height=128 style="display:inline; margin:1px"/></a>'%(link,img))
    display(HTML("".join(lines)))

In [12]:
link_templ = "https://marketplace.axieinfinity.com/profile/ronin:%s/axie/"
for client_id, elo, rank, name in answer:
    link_id = link_templ%client_id[2:]
    print(link_id, elo, rank, name)
    axieList = teams[client_id]
    axieImages = ["https://storage.googleapis.com/assets.axieinfinity.com/axies/%s/axie/axie-full-transparent.png"%x for x in axieList]
    displayAxies(axieList, axieImages, False)

https://marketplace.axieinfinity.com/profile/ronin:e15ff91aef8d953b6263993b12290439acd4dc09/axie/ 2899 77 Vergil | SW 🐳


https://marketplace.axieinfinity.com/profile/ronin:a03a27dba8ee39b651fcd8757bb0025796cde17f/axie/ 2713 323 Godzilla


https://marketplace.axieinfinity.com/profile/ronin:cb1506b8bf44fa225979dd1daf9dc6a9c7c0584b/axie/ 2713 347 Click | CNG


https://marketplace.axieinfinity.com/profile/ronin:8166b9a7368dd3fa25b077d8253b11d733e5d46a/axie/ 2711 357 youtube.com/Jenshin Ph


https://marketplace.axieinfinity.com/profile/ronin:138fefdb5117d6f37aeb39959e9a6fc516bfb834/axie/ 2704 388 Sushinobi | YGG | SAA


...

https://marketplace.axieinfinity.com/profile/ronin:3b67f5769bb34b8169190b9986d3ac97c012dd0a/axie/ 2254 7121 Spiritmonger


https://marketplace.axieinfinity.com/profile/ronin:694c993d2f71f2e4cd3c0edca4c7db915bf113d2/axie/ 2233 8127 YvanPlays


https://marketplace.axieinfinity.com/profile/ronin:686a9dc15eb518c65893dcf871576c3ca5c0220a/axie/ 2219 8845 DON Vito


https://marketplace.axieinfinity.com/profile/ronin:3faf9e5fb731a5832c1edd21332c7ca34f5b9252/axie/ 2215 9076 HeliosOm


https://marketplace.axieinfinity.com/profile/ronin:97f300359b1bd5cb7e177c37e59498e091c23e5b/axie/ 2197 10303 T axie
